# Ejercicios PySpark clase 5

In [ ]:
!pip install pyspark==3.2.0
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar
!wget https://downloads.apache.org/kafka/3.4.1/kafka_2.12-3.4.1.tgz
!tar -xzf kafka_2.12-3.4.1.tgz

In [ ]:
!pip install pyspark==3.2.0
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar
!wget https://downloads.apache.org/kafka/3.4.1/kafka_2.12-3.4.1.tgz
!tar -xzf kafka_2.12-3.4.1.tgz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 25.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805893 sha256=dffc03443345a728f7610d0bab2e70ef39d31861b7e3afb074c0c13406469da9
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
--2023-10-02 21:27:07--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... conne

## Operaciones con DataFrames en PySpark - Análisis de Videojuegos
Se proporciona un dataset de videojuegos llamado 'videogames.csv'. Realiza las siguientes operaciones utilizando PySpark:
- Carga el dataset como un DataFrame.
- Filtra los videojuegos que no pertenezcan a la plataforma "PC".
- Cuenta la cantidad de videojuegos para cada género.
- Muestra el top 5 de los géneros con más videojuegos.
- Escribe el resultado en un archivo CSV.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

def top_5_genres(spark):
    # Read the dataset
    df = spark.read.csv('videogames.csv', header=True)

    # Filter the videogames for platform 'PC'
    df = df.filter(df.Platform == 'PC')

    # Count the video games by genre
    result = df.groupBy('Genre').agg(count('*').alias('total_games'))

    # Get the top 5 genres
    result = result.sort('total_games', ascending=False).limit(5)

    # Write results to CSV
    result.write.csv('top_5_genres.csv', mode='overwrite')

## Análisis de tendencias en tweets en tiempo real
Se reciben trasmisiones de tweets por un sistema Kafka con el topic 'tweets'. Utilizando PySpark Streaming, conectarse al sistema Kafka y realizar las siguientes operaciones:
- Leer los tweets en tiempo real.
- Filtrar los tweets que contengan ciertas palabras clave, por ejemplo, "covid", "vaccine", "pandemic".
- Contar la cantidad de tweets por cada palabra clave.
- Imprimir el conteo actualizado en la consola.


In [ ]:
!./kafka_2.12-3.4.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.4.1/config/zookeeper.properties
!./kafka_2.12-3.4.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.4.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 20
!ps -ef | grep kafka
# iniciando el tópico iris con replicación 1 y 1 partición
!./kafka_2.12-3.4.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic tweets
!./kafka_2.12-3.4.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic tweets

Waiting for 10 secs until kafka and zookeeper services are up and running
root        1639       1 11 21:27 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.4.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.4.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.4.1/bin/../config/log4j.properties -cp /content/kafka_2.12-3.4.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.4.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.4.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.4.1/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.12-3.4.1/bin/../libs/commons-cli-1.4.jar:/conte

In [ ]:
from pyspark.sql.functions import split, count
from pyspark.sql.streaming import DataStreamReader

def analyze_tweets(spark):
    # Connect to Kafka and subscribe to the tweets topic
    tweets = (spark.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers", "localhost:9092")
            .option("subscribe", "tweets")
            .load())

    # Extract the tweet text
    texts = tweets.selectExpr("CAST(value AS STRING) as text")

    # Split the tweets and filter for keywords
    words = texts.select(split('text', " ").alias('word'))
    keywords = words.filter(words.word.isin(["covid", "vaccine", "pandemic"]))

    # Count the tweets for each keyword
    result = keywords.groupBy('word').count()

    # Write the output to console
    query = (result.writeStream
            .outputMode("complete")
            .format("console")
            .start())

    # Wait for query to finish
    query.awaitTermination()

## Procesamiento de imágenes en PySpark
Utiliza el ImageSchema de PySpark para realizar las siguientes operaciones:
- Leer un grupo de imágenes en un DataFrame.
- Mostrar información sobre las imágenes, como el formato, altura, anchura y el canal de color.
- Filtrar imágenes con una altura y anchura mayor a 640x480.
- Guardar las imágenes filtradas en un archivo binario.


In [ ]:
from pyspark.ml.image import ImageSchema

def process_images(spark):
    # Read images using ImageSchema
    images = ImageSchema.readImages("images")

    # Display image information
    print(images.select("image.height", "image.width",
                        "image.nChannels", "image.mode").collect())

    # Filter images based on dimensions
    big_images = images.filter('image.height > 640 and image.width > 480')

    # Save filtered images as binary files
    (big_images.write
    .parquet("big_images_parquet", mode='overwrite'))
